# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

## Solution 1
- The data represents 17 marketing campaigns

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [41]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, plot_confusion_matrix, mean_squared_error
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.inspection import permutation_importance

In [42]:
df = pd.read_csv('data/bank-additional.csv', sep = ';')

In [43]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4119 entries, 0 to 4118
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             4119 non-null   int64  
 1   job             4119 non-null   object 
 2   marital         4119 non-null   object 
 3   education       4119 non-null   object 
 4   default         4119 non-null   object 
 5   housing         4119 non-null   object 
 6   loan            4119 non-null   object 
 7   contact         4119 non-null   object 
 8   month           4119 non-null   object 
 9   day_of_week     4119 non-null   object 
 10  duration        4119 non-null   int64  
 11  campaign        4119 non-null   int64  
 12  pdays           4119 non-null   int64  
 13  previous        4119 non-null   int64  
 14  poutcome        4119 non-null   object 
 15  emp.var.rate    4119 non-null   float64
 16  cons.price.idx  4119 non-null   float64
 17  cons.conf.idx   4119 non-null   f

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4119 entries, 0 to 4118
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             4119 non-null   int64  
 1   job             4119 non-null   object 
 2   marital         4119 non-null   object 
 3   education       4119 non-null   object 
 4   default         4119 non-null   object 
 5   housing         4119 non-null   object 
 6   loan            4119 non-null   object 
 7   contact         4119 non-null   object 
 8   month           4119 non-null   object 
 9   day_of_week     4119 non-null   object 
 10  duration        4119 non-null   int64  
 11  campaign        4119 non-null   int64  
 12  pdays           4119 non-null   int64  
 13  previous        4119 non-null   int64  
 14  poutcome        4119 non-null   object 
 15  emp.var.rate    4119 non-null   float64
 16  cons.price.idx  4119 non-null   float64
 17  cons.conf.idx   4119 non-null   f

### The Business Objective
- The business objective is to predict if the client will subscribe (yes/no) a term deposit (variable y).

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [46]:
XX = df[['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'y']]
X = XX.drop(['y'], axis=1)
y = XX.y

In [47]:
le = LabelEncoder()

In [48]:
X.job = le.fit_transform(X.job)
X.marital = le.fit_transform(X.marital)
X.education = le.fit_transform(X.education)
X.default = le.fit_transform(X.default)
X.housing = le.fit_transform(X.housing)
X.loan = le.fit_transform(X.loan)

In [49]:
X

,age,job,marital,education,default,housing,loan
0,30,1,1,2,0,2,0
1,39,7,2,3,0,0,0
2,25,7,1,3,0,2,0
3,38,7,1,2,0,1,1
4,47,0,1,6,0,2,0
...,...,...,...,...,...,...,...
4114,30,0,1,1,0,2,2
4115,39,0,1,3,0,2,0
4116,27,8,2,3,0,0,0
4117,58,0,1,3,0,0,0


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.3)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [51]:
y_df = pd.DataFrame(y).value_counts(normalize=True)

In [52]:
pd.DataFrame(y_df)

,0
y,
no,0.890507
yes,0.109493


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [53]:
start = time.time()
logreg = LogisticRegression(random_state=42, class_weight='balanced').fit(X_train, y_train)
stop = time.time()
logreg_train_time = stop - start
print(f"Training time: {logreg_train_time}s")

Training time: 0.11711311340332031s


### Problem 9: Score the Model

What is the accuracy of your model?

In [54]:
logreg_train_acc = (logreg.score(X_train, y_train))
logreg_test_acc = (logreg.score(X_test, y_test))

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

**KNN Classifier**

In [55]:
from sklearn.neighbors import KNeighborsClassifier

start = time.time()
knn = KNeighborsClassifier().fit(X_train, y_train)
stop = time.time()
knn_train_time = stop - start
print(f"Training time: {knn_train_time}s")

Training time: 0.029223203659057617s


In [56]:
knn_train_acc = (knn.score(X_train, y_train))
knn_test_acc = (knn.score(X_test, y_test))

**DecisionTree Classifier**

In [57]:
from sklearn.tree import DecisionTreeClassifier
start = time.time()
dtree = DecisionTreeClassifier().fit(X_train, y_train)
stop = time.time()
dtree_train_time = stop - start
print(f"Training time: {dtree_train_time}s")

Training time: 0.033960580825805664s


In [58]:
dtree_train_acc = (dtree.score(X_train, y_train))
dtree_test_acc = (dtree.score(X_test, y_test))

**SVC Classifier**

In [59]:
from sklearn.svm import SVC
start = time.time()
svc = SVC().fit(X_train, y_train)
stop = time.time()
svc_train_time = stop - start
print(f"Training time: {svc_train_time}s")

Training time: 0.8428261280059814s


In [60]:
svc_train_acc = (svc.score(X_train, y_train))
svc_test_acc = (svc.score(X_test, y_test))

In [61]:
val_df = pd.DataFrame(columns = ['Model','Train Time','Train Accuracy','Test Accuracy'])

In [62]:
val_df.loc[len(val_df.index)] = ['Logistic Regression', logreg_train_time, logreg_train_acc, logreg_test_acc] 
val_df.loc[len(val_df.index)] = ['KNeighborsClassifier', knn_train_time, knn_train_acc, knn_test_acc] 
val_df.loc[len(val_df.index)] = ['DecisionTree Classifier', dtree_train_time, dtree_train_acc, dtree_test_acc] 
val_df.loc[len(val_df.index)] = ['SVC Classifier', svc_train_time, svc_train_acc, svc_test_acc]

val_df = val_df.reset_index()
val_df = val_df.set_index('Model')
val_df = val_df.drop('index', axis=1)

In [63]:
val_df

,Train Time,Train Accuracy,Test Accuracy
Model,,,
Logistic Regression,0.117113,0.561915,0.559871
KNeighborsClassifier,0.029223,0.895595,0.883495
DecisionTree Classifier,0.033961,0.961845,0.835761
SVC Classifier,0.842826,0.889005,0.894013


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

**LogisticRegression Tuning**

In [64]:
solvers = ['newton-cg', 'lbfgs', 'liblinear']

penalty = ['l2']
c_values = [10, 1.0, 0.1, 0.01]
grid = dict(solver=solvers,penalty=penalty,C=c_values)

logreg_grid = GridSearchCV(logreg, param_grid=grid, n_jobs=-1, scoring='accuracy',error_score=0).fit(X_train, y_train)
logreg_cv_res = logreg_grid.cv_results_

In [65]:
r = permutation_importance(logreg_grid, X_test, y_test, n_repeats = 10, random_state = 123)
pd.DataFrame({"Variable":X.columns, "Score":r.importances_mean}).sort_values(by = "Score", ascending = False)

,Variable,Score
0,age,0.005663
4,default,0.003074
3,education,0.001375
6,loan,0.000809
5,housing,-0.002265
2,marital,-0.002751
1,job,-0.007929


In [72]:
logreg_grid.best_estimator_

LogisticRegression(C=0.01, class_weight='balanced', random_state=42,
                   solver='newton-cg')

In [73]:
logreg_grid.best_score_

0.5709326256499134

In [74]:
logreg_grid.best_params_

{'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}

**KNN Tuning**

In [66]:
leaf_size = [30,20,10]

metric = ['minkowski']
n_neighbors = [10, 5, 3, 2]
grid = dict(leaf_size=leaf_size,metric=metric,n_neighbors=n_neighbors)

knn_grid = GridSearchCV(knn, param_grid=grid, n_jobs=-1, scoring='accuracy',error_score=0).fit(X_train, y_train)
knn_cv_res = knn_grid.cv_results_

In [67]:
r = permutation_importance(knn_grid, X_test, y_test, n_repeats = 10, random_state = 123)
pd.DataFrame({"Variable":X.columns, "Score":r.importances_mean}).sort_values(by = "Score", ascending = False)

,Variable,Score
0,age,0.004288
5,housing,0.001456
1,job,0.001294
6,loan,0.000890
2,marital,0.000809
3,education,0.000243
4,default,0.000243


In [68]:
knn_grid.best_estimator_

KNeighborsClassifier(n_neighbors=10)

In [69]:
knn_grid.best_score_

0.8886578085884844

In [70]:
knn_grid.best_params_

{'leaf_size': 30, 'metric': 'minkowski', 'n_neighbors': 10}

**DecisionTree Tuning**

In [76]:
splitter = ['best']

criterion = ['gini']
ccp_alphas = [10, 1.0, 0.1, 0.01]
grid = dict(splitter=splitter,criterion=criterion,ccp_alpha=ccp_alphas)

dtree_grid = GridSearchCV(dtree, param_grid=grid, n_jobs=-1, scoring='accuracy',error_score=0).fit(X_train, y_train)
dtree_cv_res = dtree_grid.cv_results_

In [77]:
r = permutation_importance(dtree_grid, X_test, y_test, n_repeats = 10, random_state = 123)
pd.DataFrame({"Variable":X.columns, "Score":r.importances_mean}).sort_values(by = "Score", ascending = False)

,Variable,Score
0,age,0.0
1,job,0.0
2,marital,0.0
3,education,0.0
4,default,0.0
5,housing,0.0
6,loan,0.0


In [78]:
dtree_grid.best_estimator_

DecisionTreeClassifier(ccp_alpha=10)

In [79]:
dtree_grid.best_score_

0.889004429039091

In [80]:
dtree_grid.best_params_

{'ccp_alpha': 10, 'criterion': 'gini', 'splitter': 'best'}

**SVC Tuning**

In [81]:
kernel = ['rbf']

decision_function_shape = ['ovr']
c_values = [10, 1.0, 0.1, 0.01]
grid = dict(kernel=kernel,decision_function_shape=decision_function_shape,C=c_values)

svc_grid = GridSearchCV(svc, param_grid=grid, n_jobs=-1, scoring='accuracy',error_score=0).fit(X_train, y_train)
svc_cv_res = svc_grid.cv_results_

In [82]:
r = permutation_importance(svc_grid, X_test, y_test, n_repeats = 10, random_state = 123)
pd.DataFrame({"Variable":X.columns, "Score":r.importances_mean}).sort_values(by = "Score", ascending = False)

,Variable,Score
0,age,0.0
1,job,0.0
2,marital,0.0
3,education,0.0
4,default,0.0
5,housing,0.0
6,loan,0.0


In [83]:
svc_grid.best_estimator_

SVC()

In [84]:
svc_grid.best_score_

0.889004429039091

In [85]:
svc_grid.best_params_

{'C': 1.0, 'decision_function_shape': 'ovr', 'kernel': 'rbf'}

#### Findings